In [1]:
import os
import pprint
import sys
import re
from xml.etree import ElementTree as ET
from datetime import datetime

In [2]:
# read all files in downloads folder, recursively
def read_xml_files(path):
    for r, _, f in os.walk(path):
        for file in f:
            if '.xml' not in file or "BOE" not in file:
                continue
            yield os.path.join(r, file)

In [46]:
METADATA_FIELDS = [
    ("identificador", None),
    ("titulo", None),
    ("departamento", None),
    ("fecha_publicacion", lambda x: datetime.strptime(x, "%Y%m%d")),
    ("origen_legislativo", None),
    ("rango", None),
]

def jsonify_boe_entry(xml):
    entry_json = {}

    # get metadata
    metadata = xml.find("metadatos")
    for tag, parser in METADATA_FIELDS:
        element = metadata.find(tag)
        if element is None: continue
        text = element.text
        entry_json[tag] = parser(text) if parser is not None else text
    
    # get topics
    entry_json["materias"] = [topic.text for topic in xml.findall(".//materia")]

    # get references
    past_refs = []
    for ref in xml.findall(".//anterior"):
        past_refs.append({
            "identificador": ref.get("referencia"),
            "texto": ref.find("texto").text,
        })
    entry_json["anteriores"] = past_refs

    future_refs = []
    for ref in xml.findall(".//posterior"):
        future_refs.append({
            "identificador": ref.get("referencia"),
            "texto": ref.find("texto").text,
        })
    entry_json["posteriores"] = future_refs

    # get XML text
    xml_text = xml.find("texto")
    html_text = ET.tostring(xml_text, encoding='utf8', method="html").decode('utf8')
    html_text = "\n".join(html_text.split("\n")[1:-1])
    entry_json["texto"] = html_text

    # get paragraphs
    entry_json["parrafos"] = [paragraph.text for paragraph in xml_text.findall(".//p")]

    return entry_json

In [48]:
for filepath in read_xml_files("../downloads/"):
    # read file
    xml = ET.parse(filepath)
    entry_json = jsonify_boe_entry(xml)
    break
    

{'anteriores': [{'identificador': 'BOE-A-2015-11430',
                 'texto': 'el art. 90.2 y 3 de la Ley del Estatuto de los '
                          'Trabajadores, texto refundido aprobado por Real '
                          'Decreto Legislativo 2/2015, de 23 de octubre'},
                {'identificador': 'BOE-A-2010-9274',
                 'texto': 'el Real Decreto 713/2010, de 28 de mayo'},
                {'identificador': 'BOE-A-2016-1047',
                 'texto': 'el Convenio publicado por Resolución de 19 de enero '
                          'de 2016'}],
 'departamento': 'Ministerio de Trabajo y Economía Social',
 'fecha_publicacion': datetime.datetime(2023, 11, 2, 0, 0),
 'identificador': 'BOE-A-2023-22481',
 'materias': ['Aeropuertos y aeródromos',
              'Convenios colectivos',
              'Transportes aéreos'],
 'origen_legislativo': 'Estatal',
 'parrafos': ['Visto el texto del V Convenio colectivo de la empresa Easyjet '
              'Handling (Spain), s

In [44]:
xml = ET.parse("../downloads/2023/10/21/BOE-A-2023-21674.xml")

In [41]:
xml.findall(".//anterior")[0]

'BOE-A-2022-18488'

In [36]:
xml.find("analisis").find("referencias").find("anteriores").findall("anterior")

[<Element 'anterior' at 0x111699da0>]